<a href="https://colab.research.google.com/github/RaoSharjeelKhan/Deep_Learning_Projects/blob/main/test_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading deepface to access useful functions


In [1]:
pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 7.4 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=b0484391ff80874190e900ec0b460a2ac7b2e9d04b48ea4deb0f52fe595272c4
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


## Unzipping the folder

In [3]:
from zipfile import ZipFile
DT="/content/Dataset3000.zip"
with ZipFile(DT, 'r') as zip:
  zip.extractall()

## Importing Required Libraries

In [8]:
import os
import time
import pickle
import numpy as np
from deepface import DeepFace
from deepface.commons import distance as dst

## Loading the file paths in a list

In [ ]:
folder_path = "/content/Dataset3000"
file_names = os.listdir(folder_path)
file_links = [os.path.join(folder_path, file_name) for file_name in file_names]
file_links_list = [ file_link  for file_link in file_links]
print("Total Image Present  :  "+ str(len(file_links_list )))


Total Image Present  :  2998


## Creating the Embedding of Images Present in the Folder

In [ ]:
embeddings = []
#counter = 0
for img_path in file_links_list:
    img_embedding=DeepFace.represent(img_path,enforce_detection=False, model_name='ArcFace')
    embeddings.append(img_embedding[0]['embedding'])
    #counter += 1
    #print(counter)
print("Embedding generation completed.")

Embedding generation completed.


## Or Loading the Existing Embedings

In [5]:
embedding_path = 'embeddings3000.pkl'
with open(embedding_path, 'rb') as pickle_file:
    face_embeddings = pickle.load(pickle_file)

name_path = 'link_list3000.pkl'
with open(name_path, 'rb') as pickle_file:
        face_links = pickle.load(pickle_file)

print("Lists loaded from pickle files.")

Lists loaded from pickle files.


## Defining A Verification function

In [6]:
def verify_faces(img1_emb, img2_emb, model_name="ArcFace",  distance_metric="cosine"):
    distances = []
    img1_representation = img1_emb[0]['embedding']
    img2_representation = img2_emb
    distance = dst.findCosineDistance(img1_representation, img2_representation)
    distances.append(distance)
    threshold = dst.findThreshold(model_name, distance_metric)
    best_distance = min(distances)
    return {
        "verified": best_distance <= threshold,
        "distance": best_distance,
        "threshold": threshold,
        "model": model_name,
    }

## Face Verification

In [9]:
start_time = time.time()
test_image_path = '/content/Arnold.PNG'
test_image_emb = DeepFace.represent(test_image_path, model_name="ArcFace")
embeddings_array = np.array(face_embeddings)
cosine_similarities = np.dot(test_image_emb[0]['embedding'], embeddings_array.T)
top_indices = np.argsort(cosine_similarities)[-5:][::-1]
print("Results:")
for idx in top_indices:
    similarity_score = cosine_similarities[idx]
    if similarity_score > 7:
        obj = verify_faces(test_image_emb, embeddings_array[idx], model_name='ArcFace')
        if obj['verified'] and obj['distance'] < 0.6:
            print("\nMatched with:")
            print(f"Matching Person: {os.path.basename(face_links[idx])}")
            print(f"Distance: {obj['distance']}")
end_time = time.time()
total_time = end_time - start_time
print(f"\nTime: {total_time:.2f} seconds")

Results:

Matched with:
Matching Person: Arnold_Schwarzenegger_0014.jpg
Distance: 0.27676248384209146

Time: 0.40 seconds


## Dumping lists as a pickle file

In [ ]:
embeddings_file_path = 'embeddings.pkl'
name_file_path = 'name_list.pkl'

with open(embeddings_file_path, 'wb') as file:
    pickle.dump(embeddings, file)

with open(name_file_path, 'wb') as file:
    pickle.dump(file_links_list, file)

print("Lists saved to as a pickle file.")

Lists saved to as a pickle file.
